# Practica 4
## Captcha Destroyer hecho por: Python Haters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán

# Import


In [ ]:
import string
from tensorflow.keras import layers
from PIL import Image
import numpy as np
import string
import os

## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos, para esto usaremos PIL

Cogemos la direcion de nuestro dataset

In [ ]:
directorio_input = 'samples/'
directorio_output = 'samples_transform/'

# Definimos el tamanio de nuestras imagenes
imgshape=(50,200,1) #50-alto, 200-ancho, 1-nym canales

Primero necesitamos saber cuales son los caracteres que nos podemos encontrar en los 'catcha' y, ya que estamos, la cantidad de caracteres que son.

In [ ]:
symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
numChar = len(symbols) # numero de chars posibles

Segundo escalaremos las imagenes a 200 de ancho y 50 de alto

In [ ]:
def convertir_imagen(ruta_input, ruta_output, dimension=(200, 50)):

    # Abrir imagen
    imagen = Image.open(ruta_input)
    
    # Cambiar tamaño
    imagen = imagen.resize(dimension)
    
    # Convertir a blanco y negro
    imagen_bn = imagen.convert('L')  # 'L' es modo de 8 bits en escala de grises
    
    # Guardar la nueva imagen
    imagen_bn.save(ruta_output)

Ahora convertinemos las imagenes en matrices de 0 y 1, es decir en Blanco = 0 y negro = 1

In [ ]:
def imagen_a_matriz(ruta_imagen, umbral=128):

    # Abre y convierte a escala de grises
    imagen = Image.open(ruta_imagen)
    
    # Aplica umbral para binarizar (blanco o negro)
    imagen_bn = imagen.point(lambda p: 255 if p > umbral else 0)
    
    ancho, alto = imagen_bn.size
    pixeles = imagen_bn.load()
    
    matriz = []
    for y in range(alto):
        fila = []
        for x in range(ancho):
            pixel = pixeles[x, y]
            valor = 1 if pixel == 0 else 0  # Negro -> 1, Blanco -> 0
            fila.append(valor)
        matriz.append(fila)
    
    return matriz

Por ultimo, sabemos que el nombre de la imagen es la solucion del catchap, asique ya lo guardaremos los targets

In [ ]:
def getTargets():
    
    letersImg_array = np.zeros((5, ,numChar)) # 5 (letras de catchap) * n = 1070 (tamanio del dataset ) * 36 (posibles caracteres)  (con todas las entradas a 0)

    for i, pic in enumerate(os.listdir(directorio_input)):
    # i : indice en el que se encuentra la imagen en el directorio (0-(n-1))->(n)
    # pic : contiene el nombre de la imagen en posicion i, que es equivalente a la solucion del catchap + .png
       
        pic_target = pic[:-4]# guardamos el objetimo (solucion del catchap), que es quivalente a el nombre - .png (.png = 4bits => -4)

        if len(pic_target) < 6: #solo captcha de 5 letras o menos

            target=np.zeros((5,numChar)) #5 (letras de catchap) * 36 (posibles caracteres)  (con todas las entradas a 0) = (ej: 0:0,0,....(36),0; 0:0,0,...)

            for j, k in enumerate(pic_target):
            # j : indice en el que se encuentra la palabra (0-4)->(5)
            # k : letra correspondiente a la pos j
                indexSymbol = symbols.find(k) # indexSymbol guarda la pos de la letra k, respecto al array de  symbols
                target[j, indexSymbol] = 1 # en la posicion j : remplazo en la pos index el 0 por un 1 (ej: en la pos j esta la letra m)

            letersImg_array[:,i] = target # guardo el target (solucion) en pos i

    return letersImg_array

Definimos una funcion auxiliar para mostrar matriz, para depurar

In [ ]:
def mostrar_matriz(matriz, ancho=200, alto=50):
    for c in range(0, alto):
        for f in range(0, ancho):
            print(matriz[c][f], end="")
        print('\n')
    print('\n')


In [ ]:

# Convertir 'test' imagenes a blanco y negro y tamaño 200x50
DEBUG = True
test = 1

for nombre_archivo in os.listdir(directorio_input):

    # Convertir todas las imagenes del directorio a blanco y negro y dimension establecidos
    archivo_original = directorio_input + nombre_archivo
    archivo_nuevo = directorio_output + nombre_archivo
    convertir_imagen(archivo_original, archivo_nuevo)
    matriz = imagen_a_matriz(archivo_nuevo)
 
    if(DEBUG):
        print(f"Archivo antiguo: {archivo_original}")
        print(f"Archivo nuevo: {archivo_nuevo}\n")
        mostrar_matriz(matriz)

    test = test - 1 # No petar el  PC
    if(test == 0):
        break

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual. (logic CNN)

In [ ]:
#create model
def createmodel():
    img = layers.Input(shape=imgshape) # Get image as an input of size 50,200,1
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img) #50*200
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 25*100
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 13*50
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3) #to improve the stability of model
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 7*25
    
    flat = layers.Flatten()(mp3) #convert the layer into 1-D

    outs = []
    for _ in range(5): #for 5 letters of captcha
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1) #drops 0.5 fraction of nodes
        res = layers.Dense(nchar, activation='sigmoid')(drop)

        outs.append(res) #result of layers
    
    # Compile model and return it
    model = Model(img, outs) #create model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

## Parte 2: Reconocer una cadena de dígitos y letras bien separadas
Ahora que sabemos identificar letras y números por separado podemos agruparlos bien separados en una cadena para leerlos y escribirlos en orden de aparición. (logic RNN)

## Parte 3: Reconocer dígitos y letras que se juntan/mezclan
Por último queda ser capaces de distinguir los números y letras que están solapados en la cadena para poder escribirlos (CRNN = CNN + RNN)